In [48]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [49]:

# Load the data
population = pd.read_csv("Population.csv")

# Clean column names: lowercase, replace hyphens with underscores
population.columns = (
    population.columns
    .str.lower()
    .str.replace("-", "_")
    .str.replace(" ", "_")
)

# Show cleaned columns
print("Cleaned columns:", population.columns.tolist())

# Confirm data types and nulls (should be fine)
print(population.info())
print(population.isnull().sum())

Cleaned columns: ['sa2_code', 'sa2_name', '0_4_people', '5_9_people', '10_14_people', '15_19_people', '20_24_people', '25_29_people', '30_34_people', '35_39_people', '40_44_people', '45_49_people', '50_54_people', '55_59_people', '60_64_people', '65_69_people', '70_74_people', '75_79_people', '80_84_people', '85_and_over_people', 'total_people']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sa2_code            373 non-null    int64 
 1   sa2_name            373 non-null    object
 2   0_4_people          373 non-null    int64 
 3   5_9_people          373 non-null    int64 
 4   10_14_people        373 non-null    int64 
 5   15_19_people        373 non-null    int64 
 6   20_24_people        373 non-null    int64 
 7   25_29_people        373 non-null    int64 
 8   30_34_people        373 non-null    int64 
 9   35_39_people  

In [50]:
# This loads the POIs from the REST service directly
poi_url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query?where=1%3D1&outFields=*&f=geojson"

pois = gpd.read_file(poi_url)
print(pois.columns)
pois.head()

Index(['objectid', 'topoid', 'poigroup', 'poitype', 'poiname', 'poilabel',
       'poilabeltype', 'poialtlabel', 'poisourcefeatureoid', 'accesscontrol',
       'startdate', 'enddate', 'lastupdate', 'msoid', 'centroidid',
       'shapeuuid', 'changetype', 'processstate', 'urbanity', 'geometry'],
      dtype='object')


,objectid,topoid,poigroup,poitype,poiname,poilabel,poilabeltype,poialtlabel,poisourcefeatureoid,accesscontrol,startdate,enddate,lastupdate,msoid,centroidid,shapeuuid,changetype,processstate,urbanity,geometry
0,1,500000000,9,Mine - Underground,None,Mine - Underground,GENERIC,None,157,1,1628668563000,32503680000000,1628668617000,233046,None,729e2b57-0cd4-3f70-90fa-9dce09e34a8e,I,None,S,POINT (152.12202 -31.10616)
1,2,500005504,3,Lookout,KUNDERANG LOOKOUT,KUNDERANG LOOKOUT,NAMED,None,56,1,1285588392000,32503680000000,1285588392535,83091,None,d88a28a8-c572-3992-995f-d26a274aea18,I,None,S,POINT (152.29869 -31.02148)
2,3,500005505,3,Lookout,FALLS LOOKOUT,FALLS LOOKOUT,NAMED,None,56,1,1285588392000,32503680000000,1285588392535,83691,None,21b476d2-6519-3e28-8b19-1526fcb9652f,I,None,S,POINT (152.33786 -31.01576)
3,4,500005507,3,Lookout,MCCOYS LOOKOUT,MCCOYS LOOKOUT,NAMED,None,56,1,1285588392000,32503680000000,1285588392535,83380,None,016d69b2-6530-39e7-89a6-6e3054df55ac,I,None,S,POINT (152.34181 -31.01897)
4,5,500012781,3,Picnic Area,WILSON RIVER PICNIC AREA,WILSON RIVER PICNIC AREA,NAMED,None,62,1,1608714678000,32503680000000,1608714706360,231054,None,49ad26c8-609e-3aa0-b4ad-51459b43ab51,M,None,S,POINT (152.47882 -31.20754)


In [51]:
# Load stops.txt
stops = pd.read_csv('Stops.txt')

# Preview
stops.head()

# Some cleaning
# Drop rows where stop_lat or stop_lon is missing
stops = stops.dropna(subset=['stop_lat', 'stop_lon'])

# We want to only select the stops inside three areas: 
# Inner West, North Sydney and Hornsby, City and Inner South

# No column name telling which region a stop belongs to
# So: use the stop's latitude and longitude (stop_lat, stop_lon) to filter our disired data

In [52]:
sa2 = gpd.read_filesa2 = gpd.read_file("SA2_2021_AUST_SHP_GDA2020\\SA2_2021_AUST_GDA2020.shp")


print(sa2.columns)
sa2.head()

sa2_sydney = sa2[sa2['GCC_NAME21'] == 'Greater Sydney']

# Create geometry column for stops
stops['geometry'] = stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)

# Turn stops into a GeoDataFrame (same CRS)
stops_gdf = gpd.GeoDataFrame(stops, geometry='geometry', crs="EPSG:4283")

stops_gdf = stops_gdf.to_crs(epsg=7844)

stops_with_sa2 = gpd.sjoin(stops_gdf, sa2_sydney, how='left', predicate='within')

# print(stops_with_sa2.head(10)) #debug

target_regions = [
    'Sydney - Inner West',
    'Sydney - North Sydney and Hornsby',
    'Sydney - City and Inner South'
]

filtered_stops = stops_with_sa2[stops_with_sa2['SA4_NAME21'].isin(target_regions)]

filtered_stops = filtered_stops.dropna(subset=['SA4_NAME21'])

columns_to_keep = [
    'stop_id', 'stop_name', 'stop_lat', 'stop_lon',
    'wheelchair_boarding', 'SA2_CODE21', 'SA2_NAME21', 'SA4_NAME21', 'geometry'
]

filtered_stops = filtered_stops[columns_to_keep]

filtered_stops.columns = filtered_stops.columns.str.lower()

filtered_stops.to_csv('s_filtered.csv', index=False)


Index(['SA2_CODE21', 'SA2_NAME21', 'CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21',
       'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'GCC_CODE21', 'GCC_NAME21',
       'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'AREASQKM21',
       'LOCI_URI21', 'geometry'],
      dtype='object')


In [53]:
# Adjust path as needed
catchments_primary = gpd.read_file("catchments\\catchments_primary.shp")

# Preview structure
print(catchments_primary.columns)
catchments_primary.head()

Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry'],
      dtype='object')


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.93564 -33.71612, 150.93715 -33.7..."
1,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.18336 -33.74748, 151.18443 -33.7..."
2,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.04518 -33.77303, 151.04526 -33.7..."
3,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.92567 -33.7296, 150.92602 -33.72..."
4,3918,PRIMARY,Killara PS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.15379 -33.75586, 151.15404 -33.7..."


In [54]:
# Adjust path as needed
catchments_secondary = gpd.read_file("catchments\\catchments_secondary.shp")

# Preview structure
print(catchments_secondary.columns)
catchments_secondary.head()

Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry'],
      dtype='object')


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,8503,HIGH_COED,Billabong HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.67182 -35.31444, 146.6893 -35.31..."
1,8266,HIGH_COED,James Fallon HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((147.08734 -35.86271, 147.10413 -35.8..."
2,8505,HIGH_COED,Murray HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.81448 -35.78341, 146.8125 -35.79..."
3,8458,HIGH_COED,Kingswood HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"MULTIPOLYGON (((150.686 -33.74031, 150.68631 -..."
4,8559,HIGH_COED,Jamison HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((150.69513 -33.75627, 150.68936 -33.7..."


In [55]:
# Adjust path as needed
catchments_future = gpd.read_file("catchments\\catchments_future.shp")

# Preview structure
print(catchments_future.columns)
catchments_future.head()

Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'geometry'],
      dtype='object')


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,geometry
0,8416,HIGH_COED,Ku-ring-gai HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.19849 -33.5399, 151.19945 -33.54..."
1,8161,HIGH_BOYS,Randwick BHS,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.27152 -33.91402, 151.27152 -33.9..."
2,8539,HIGH_COED,SSC Blackwattle Bay,20220609,0,0,0,0,0,0,0,0,0,0,0,2024,2024,"POLYGON ((151.15292 -33.83939, 151.16144 -33.8..."
3,8400,HIGH_COED,St Ives HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.17794 -33.6982, 151.17859 -33.69..."
4,8555,HIGH_COED,Rose Bay SC,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.28072 -33.83287, 151.28095 -33.8..."


In [56]:
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd

In [57]:


credentials = os.path.expanduser("\\Users\\uwhit\\data2001Tut\\Credentials.json")


def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = "sydney_analysis"  ##### The right database
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [58]:
db, conn = pgconnect(credentials)

Connected successfully.


In [59]:
import geopandas as gpd

# Define shapefile paths
base_path = "\\Users\\uwhit\\DATA2001-A\\catchments"
shapefiles = {
    "catchments_primary": "catchments_primary.shp",
    "catchments_secondary": "catchments_secondary.shp",
    "catchments_future": "catchments_future.shp"
}

# Loop through the catchment files and upload to PGadmin
for table_name, shp_file in shapefiles.items():
    print(f"Processing {table_name}...")
    gdf = gpd.read_file(os.path.expanduser(os.path.join(base_path, shp_file)))
    gdf.columns = [col.lower() for col in gdf.columns]  
    gdf.to_postgis(name=table_name, con=db, if_exists="replace", index=False)
    print(f"✅ Uploaded {table_name} successfully.")

print(f"Using DB: {db.url.database}")

Processing catchments_primary...
✅ Uploaded catchments_primary successfully.
Processing catchments_secondary...
✅ Uploaded catchments_secondary successfully.
Processing catchments_future...
✅ Uploaded catchments_future successfully.
Using DB: sydney_analysis


In [60]:
sa2_path = "SA2_2021_AUST_SHP_GDA2020\\SA2_2021_AUST_GDA2020.shp"
sa2_gdf = gpd.read_file(os.path.expanduser(sa2_path))

#Keep only useful columns
sa2_gdf.columns = [col.lower() for col in sa2_gdf.columns]
columns_to_keep = ["sa2_code21", "sa2_name21", "sa4_name21", "geometry"]
sa2_gdf = sa2_gdf[columns_to_keep]

# Filter rows by target SA4 regions
target_sa4 = [
    "Sydney - Inner West",
    "Sydney - North Sydney and Hornsby",
    "Sydney - City and Inner South"
]
sa2_gdf = sa2_gdf[sa2_gdf["sa4_name21"].isin(target_sa4)]

# SUpload to PostgreSQL
sa2_gdf.to_postgis(name="sa2_regions", con=db, if_exists="replace", index=False)

print(f"✅ Cleaned and uploaded 'sa2_regions' with {len(sa2_gdf)} rows.")
sa2_gdf.head()

✅ Cleaned and uploaded 'sa2_regions' with 74 rows.


,sa2_code21,sa2_name21,sa4_name21,geometry
343,117011320,Banksmeadow,Sydney - City and Inner South,"POLYGON ((151.20807 -33.95405, 151.20817 -33.9..."
344,117011321,Botany,Sydney - City and Inner South,"POLYGON ((151.18965 -33.94813, 151.18919 -33.9..."
345,117011323,Pagewood - Hillsdale - Daceyville,Sydney - City and Inner South,"POLYGON ((151.22312 -33.92869, 151.22189 -33.9..."
346,117011324,Port Botany Industrial,Sydney - City and Inner South,"POLYGON ((151.22091 -33.96895, 151.22066 -33.9..."
347,117011325,Sydney Airport,Sydney - City and Inner South,"POLYGON ((151.17103 -33.927, 151.17167 -33.926..."


# Task 2 - North Sydney & Hornsby

In [62]:
northSydney_gdf = gpd.read_postgis("""
    SELECT *
    FROM sa2_regions
    WHERE sa4_name21 = 'Sydney - North Sydney and Hornsby'
""", conn, geom_col="geometry")

In [63]:
import requests
import io
import geopandas as gpd

def get_pois_from_bbox(minx, miny, maxx, maxy):
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    params = {
        "f": "geojson",
        "geometryType": "esriGeometryEnvelope",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "inSR": "4283",
        "outSR": "4283"
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return gpd.read_file(io.StringIO(response.text))
    except Exception as e:
        print(f"❌ Error fetching POIs for bbox {minx},{miny},{maxx},{maxy}:", e)
        return None


In [64]:
import time
from shapely.geometry import shape

all_pois = []

for _, row in northSydney_gdf.iterrows():
    sa2 = row["sa2_name21"]
    minx, miny, maxx, maxy = row["geometry"].bounds
    print(f"📦 Fetching POIs for {sa2}...")
    
    pois = get_pois_from_bbox(minx, miny, maxx, maxy)
    if pois is not None and not pois.empty:
        pois["sa2_name"] = sa2
        all_pois.append(pois)
    
    time.sleep(1)  # be nice to the server


📦 Fetching POIs for Chatswood (West) - Lane Cove North...
📦 Fetching POIs for St Leonards - Naremburn...
📦 Fetching POIs for Artarmon...
📦 Fetching POIs for Castle Cove - Northbridge...
📦 Fetching POIs for Chatswood - East...
📦 Fetching POIs for Greenwich - Riverview...
📦 Fetching POIs for Lane Cove...
📦 Fetching POIs for Willoughby...
📦 Fetching POIs for Asquith - Mount Colah...
📦 Fetching POIs for Berowra - Brooklyn - Cowan...
📦 Fetching POIs for Normanhurst - Thornleigh - Westleigh...
📦 Fetching POIs for Hornsby - East...
📦 Fetching POIs for Hornsby - West...
📦 Fetching POIs for Wahroonga (West) - Waitara...
📦 Fetching POIs for Gordon - Killara...
📦 Fetching POIs for Lindfield - Roseville...
📦 Fetching POIs for Pymble...
📦 Fetching POIs for St Ives...
📦 Fetching POIs for Turramurra...
📦 Fetching POIs for Wahroonga (East) - Warrawee...
📦 Fetching POIs for Cremorne - Cammeray...
📦 Fetching POIs for Crows Nest - Waverton...
📦 Fetching POIs for Neutral Bay - Kirribilli...
📦 Fetching POI

In [65]:
from geopandas import GeoDataFrame

if all_pois:
    all_pois_gdf = gpd.GeoDataFrame(pd.concat(all_pois, ignore_index=True), crs="EPSG:4283")
    all_pois_gdf.to_postgis("pois_northSydney", con=db, if_exists="replace", index=False)
    print(f"✅ Uploaded {len(all_pois_gdf)} POIs.")
else:
    print("⚠️ No POIs collected.")

✅ Uploaded 5650 POIs.


# Task 3 - North Sydney & Hornsby

In [97]:
try:
    pois_df = gpd.read_postgis('SELECT * FROM "pois_northSydney"', conn, geom_col='geometry')
    # Group POIs by SA2 name and count them
    pois_per_sa2 = pois_df.groupby('sa2_name').size().reset_index(name='num_pois')
except Exception as e:
    print("⚠️ Query failed:", e)
    conn.rollback()

# This code attempts to load all points of interest (POIs) from the 'pois_northSydney' table in a PostGIS database.
# It then groups the POIs by their corresponding SA2 area name and counts how many POIs are in each area.
# If the query fails (e.g., due to connection or SQL errors), it prints a warning and rolls back the transaction.

In [99]:
combined = sa2_gdf.merge(stops_per_sa2, on='sa2_code21', how='left')
combined = combined.merge(pois_per_sa2, left_on='sa2_name21', right_on='sa2_name', how='left')
combined = combined.fillna(0)

# This code merges the SA2 geographic data (sa2_gdf) with stop counts and POI counts.
# - First, it joins stop counts using the 'sa2_code21' as the common key.
# - Then, it joins POI counts using the SA2 name fields from both dataframes.
# - 'how="left"' ensures all SA2 areas are retained, even if they have no stops or POIs.
# - Missing values (i.e., areas with no stops or POIs) are filled with 0.

In [115]:
# Ensure both are strings before merging
combined['sa2_code21'] = combined['sa2_code21'].astype(str)
pop_df['sa2_code21'] = pop_df['sa2_code21'].astype(str)

# Now merge
combined = combined.merge(pop_df, on='sa2_code21', how='left')

# Filter population
combined = combined[combined['tot_p_p'] >= 100]

# This code ensures that the 'sa2_code21' column in both dataframes is treated as a string,
# so they can be accurately matched during the merge.
# It then merges population data (pop_df) into the combined dataframe based on SA2 codes.
# Finally, it filters out SA2 areas with a total population ('tot_p_p') of less than 100,
# keeping only significantly populated areas for analysis.


In [117]:
for col in ['num_stops', 'num_pois']:  # add other metrics here
    mean = combined[col].mean()
    std = combined[col].std()
    combined[f'z_{col}'] = (combined[col] - mean) / std
# This code standardizes the 'num_stops' and 'num_pois' columns by converting them to z-scores.
# For each metric:
# - It calculates the mean and standard deviation.
# - Then, it computes the z-score for each SA2 area using the formula: (value - mean) / std.
# - The resulting standardized values are stored in new columns prefixed with 'z_'.
# This helps compare values across different metrics on a common scale.

In [119]:
# Sum of z-scores
z_columns = [col for col in combined.columns if col.startswith('z_')]
combined['z_sum'] = combined[z_columns].sum(axis=1)

# Sigmoid function
import numpy as np
combined['final_score'] = 1 / (1 + np.exp(-combined['z_sum']))

# This code first calculates the total z-score for each SA2 area by summing all individual z-score columns (e.g., z_num_stops, z_num_pois).
# Then, it applies the sigmoid function to the summed z-score:
# - The sigmoid function maps the z_sum to a value between 0 and 1.
# - This transforms the combined score into a normalized "final_score", making it easier to interpret or rank.
# Areas with higher positive z_sums get scores closer to 1, while lower ones approach 0.

In [121]:
final_columns = ['sa2_code21', 'sa2_name21'] + z_columns + ['z_sum', 'final_score']
final_df = combined[final_columns]

final_df.to_csv('sa2_scores_north_sydney.csv', index=False)
# This code selects the relevant columns to include in the final output:
# - It includes the SA2 code and name, all z-score columns, the total z-score ('z_sum'), and the normalized 'final_score'.
# - It creates a new DataFrame (final_df) with just these columns.
# - Finally, it exports the result to a CSV file named 'sa2_scores_north_sydney.csv' without writing row indices.
# This file contains the final scoring output for each SA2 area in North Sydney.
